In [2]:
import pandas as pd
import os
import numpy as np
import sklearn.metrics

In [3]:
url = 'https://raw.githubusercontent.com/MengSui0218/CS668-2021-MS/main/clean_data.csv'
df = pd.read_csv(url)
df = df.drop(['Unnamed: 0'], axis =1 )
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,0,No Deposit,304.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
1,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,0,No Deposit,240.0,0,Transient,98.0,0,1,Check-Out,2015-07-03
2,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,0,No Deposit,240.0,0,Transient,98.0,0,1,Check-Out,2015-07-03
3,Resort Hotel,0,9,2015,July,27,1,0,2,2,...,0,No Deposit,303.0,0,Transient,103.0,0,1,Check-Out,2015-07-03
4,Resort Hotel,1,85,2015,July,27,1,0,3,2,...,0,No Deposit,240.0,0,Transient,82.0,0,1,Canceled,2015-05-06


In [4]:
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

1. based on EDA, cancellation from years to years keeps the same level, but it dose vary in different month. Thus, in ML process, column arrival_date_year will be dropped. 
2. column related to after check-in are supposed to be dropped, such as *assigned_room_type*, *reservation_status*, 

# Prepare training data 


In [5]:
df_ml = df.drop(['arrival_date_year','reservation_status','assigned_room_type','assigned_room_type'],axis=1)

x, y = df_ml.drop(['is_canceled'], axis= 1), df_ml['is_canceled']


# Organize training steps 
In this project, there are a few category atributes that can not be numeriazed by related machine learning algorithms. Thus transdering them to number is necessery for them before applying machine learning algorithms. 
When trasfering string type to number type, using sequenced number to present a series of category string is common transformantion. But issues can arise when original categories are norminal. Simply using number to reprent norminal data will mistakely assump that categories has order and different impact level on final prediciton. Thus, we used onehot encoder to transform category data. 
OneHot Encoder adds new coresponding columns to present categories, and use "0" and "1" to present observations belongs to these categories. 

In [6]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

str_transformer = Pipeline(steps= [('onehot',OneHotEncoder(handle_unknown = 'ignore'))])
str_col = ['hotel','arrival_date_month','meal','country','market_segment','distribution_channel','deposit_type','customer_type']
preprocesser = ColumnTransformer(transformers= [('Str',str_transformer, str_col)])

# Try out different algorithms 
Try out different algorithms at the same time to select out the algorithm that has the highest prediction accuracy even based on defaut parameters. It is efficient to select the best suitable machine learning algorithms.

SVM is out of my computation resource

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfolds = 10 
split = KFold(n_splits = kfolds, shuffle = True, random_state=42)


base_model = [('dt',DecisionTreeClassifier(random_state=42)),
              ('rf', RandomForestClassifier(max_depth=2, random_state=42)),
              ('SVM',SVC(gamma='auto')),
              ('NB',GaussianNB()),
              ('KNN',KNeighborsClassifier(n_neighbors=3))]


for name, model in base_model:
  model_steps = Pipeline(steps = [('preprocesser', preprocesser),
                                  ('model', model)])
  
  cv_result = cross_val_score(model_steps,
                              x,y,
                              cv = split,
                              scoring = 'accuracy')
  
  min_score = round(min(cv_result),4)
  max_score = round(max(cv_result),4)
  mean_score = round(np.mean(cv_result),4)
  std = round(np.std(cv_result),4)
  
  print(f'{name} cross validation score: {mean_score}+/-{std}, min: {min_score}, max: {max_score}')

NameError: name 'Pipeline' is not defined

# Fine Tune the model 
1. 64% data is used to training model 
2. 33% data is used to test

# Prediction 